In [91]:
import torch
from types import SimpleNamespace
from datasets import load_dataset
from fastprogress import master_bar, progress_bar
from torchvision.transforms.v2 import RandomCrop

In [94]:
torch.set_float32_matmul_precision('high')
device="cuda:3"
audioset = load_dataset("danjacobellis/audioset_opus_24kbps",split='train').with_format("torch")
dataset_train = audioset.select(range(1000000))
dataset_valid = audioset.select(range(1000000,1040000))

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/117 [00:00<?, ?it/s]

In [38]:
config = SimpleNamespace()
# Training and optimizer config
config.batch_size = 128
config.steps_per_epoch = dataset_train.num_rows//config.batch_size
config.grad_accum_steps = 1
config.max_lr = (config.batch_size/128)*6e-4
config.min_lr = config.max_lr/100
config.plot_update = 64
config.epochs = 200
config.lr_scale = 40
config.lr_offset = 0.25
config.lr_pow = 6
config.weight_decay = 0.
config.num_workers = 12
config.audio_len = 483840
config.crop_size = 512*512
config.channels = 1

In [102]:
rand_crop = RandomCrop(
    size=(1,config.crop_size),
    pad_if_needed=True,
    fill=0
)
def train_collate_fn(batch):
    B = len(batch)
    x = torch.zeros((B, config.channels, 1, config.crop_size), dtype=torch.float)
    y = []
    for i_sample, sample in enumerate(batch):
        y.append(sample['label'])
        x[i_sample,:,:,:] = rand_crop(sample['opus']['array'].unsqueeze(0).unsqueeze(0))
    return x[:,:,0,:], y

In [ ]:
mb = master_bar(range(config.epochs))
mb.names = ['per batch','smoothed']
for i_epoch in mb:
    dataloader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=train_collate_fn
    )
    pb = progress_bar(dataloader_train, parent=mb)
    for i_batch, (x,y) in enumerate(pb):
        x = x.to(device)